In [ ]:
import sys, os

sys.path.append("../bright_objects_masks")
import radius_study
import warnings
import numpy as np

warnings.filterwarnings("ignore")

In this notebook, we show how to compute the density ratio as a function of radius around stars.  
As a reminder, we have density_ratio(r) = density(r)/mean_catalog_density. 

Basic way to compute density_ratio(r). We need to configure a configuration file stored in the config folder. To do so you can use the gen_config.py

In [ ]:
curr_dir = os.getcwd()
critical_radius = radius_study.Critical_radius(
    config_file=curr_dir + "/../config/config_examples.yaml"
)

First, we calculate the density ratio in bins of theta defined in the config file. If the lenght of the tract_list is greater than two, then multiprocessing is applied to calculate density ratio tract by tract before merging it.

In [ ]:
density_glob = critical_radius.get_density_ratio()

In [ ]:
print(density_glob)

Get the binning used for the calculations

In [ ]:
theta_bins = critical_radius.theta_bins

If density_ratio is calculated, it can directly be given to get_critical_radius. Otherwize the method will do the calculation by itself.

"get_critical_radius" calculates the radius of the circle within the cut will be performed depending on the brigthness of the star.

Here we give critical density but it is also an argument of config file

In [ ]:
critical_radius_value = critical_radius.get_critical_radius(
    density_ratio=density_glob, critical_density=0.9
)

In [ ]:
print(critical_radius_value)

Here you can see the densityratio profile for the two selected bins (red vertical line is the cut radius). Profiles might look weird on single tracts due to lack of statistics but it gets smoother when you calculate it on several tracts

In [ ]:
import matplotlib.pyplot as plt

for i in range(len(density_glob)):
    plt.close()
    plt.plot(
        theta_bins,
        density_glob[i],
        label="d(r,mag)",
        linestyle="",
        marker="+",
        markersize=5,
    )
    plt.axvline(
        critical_radius_value[i],
        color="red",
        linestyle="--",
        label=f'r_lim={critical_radius_value[i]}"',
    )
    plt.axhline(
        0.9,
        color="black",
        linestyle="--",
        label=f"d_crit={0.9}",
    )
    plt.xlabel("radius [arcsec]")
    plt.ylabel("density_ratio")
    plt.xscale("log")
    plt.legend()
    plt.show()

For further studies and modularity, we can do masks with one radius for each star but we need a different configuration. Here we split the sample in two bins (10<mag_i_truth<11 and 11<mag_i_truth<12)

In [ ]:
critical_radius = radius_study.Critical_radius(
    config_file=curr_dir + "/../config/config_examples_unique_stars.yaml"
)

In [ ]:
ra, dec, unique_density_ratio = critical_radius.get_unique_density_ratio()

In [ ]:
print(unique_density_ratio)

Same as before : we can easily get critical radius values for each star

In [ ]:
ra_l, dec_l, crit_radius_l = [], [], []
for i in range(len(unique_density_ratio)):
    ra2, dec2, unique_crit_radius = critical_radius.get_unique_critical_radius(
        ra=ra, dec=dec, density_ratio=unique_density_ratio[i]
    )
    ra_l.append(ra2)
    dec_l.append(dec2)
    crit_radius_l.append(unique_crit_radius)

In [ ]:
crit_radius_l

As there are many stars we won't show every profile here but we can make a distribution of radius

In [ ]:
binned_quantity = critical_radius.binned_quantity
bins = critical_radius.bins

In [ ]:
for i in range(len(crit_radius_l)):
    plt.hist(
        crit_radius_l[i],
        bins=np.logspace(np.log10(min(theta_bins)), np.log10(max(theta_bins)), 25),
        histtype="step",
        label=f"{bins[i]} < {binned_quantity} < {bins[i+1]}",
    )
    plt.xscale("log")
    plt.xlabel(r"$\theta_{crit}$['']", fontsize=13)
    plt.ylabel("Number of stars", fontsize=13)
    plt.title("Distribution of radiuses for two bins of bright stars", fontsize=15)
    plt.legend()